# Applied Data Science Capstone Project (Week 3)
## Segmenting and Clustering Neighborhoods in Toronto

In [97]:
# Import modules
import numpy as np
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cmap
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium

## Read the table with pandas and load as a DataFrame

In [98]:
# Scrape data from Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_alldata = pd.read_html(url, header=0)[0]
df_alldata.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Remove Boroughs that are 'Not assigned'

In [99]:
#Remove Boroughs that are 'Not assigned'
df_canada = df_alldata[df_alldata.Borough != 'Not assigned'].reset_index(drop = True)
df_canada.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## If more than one neighborhood exist in one postal code area, combined them into one row with the neighborhoods separated with commas

## Once combined, remove the duplicates 

In [100]:
# combine neighborhoods in same postal code in one row with the neighborhoods separated with commas
df_canada["Neighbourhood"] = df_canada.groupby("Postal Code")["Neighbourhood"].transform(lambda ii: ', '.join(ii))

#remove duplicates
df_canada = df_canada.drop_duplicates()
df_canada.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough

In [101]:
# cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
if sum(df_canada["Neighbourhood"] == "Not assigned")==0:
    print("No neighborhood with \"Not assigned\" status")
else:
    for idx, row in df_canada.iterrows():
        if row["Neighbourhood"] == "Not assigned":
            df_canada.at[idx, "Neighbourhood"] == row["Borough"]
    print("Filled Neighborhood info")

No neighborhood with "Not assigned" status


## Print DataFrame shape

In [102]:
# print DataFrame shape
df_canada.shape

(103, 3)

# Get the latitude and the longitude coordinates of each neighborhood to use with FourSquare API

## get the geospatial Coordinates Data from https://cocl.us/Geospatial_data

In [103]:
# get the location data and load in a DataFrame
url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geocoord=pd.read_csv(url)
df_geocoord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge two dataframes based on 'PostalCode' 

In [104]:
df_canada = pd.merge(df_canada, df_geocoord, on = 'Postal Code')
df_canada.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Filter the DataFrame for Canada to keep only the rows in which "Borough" contains the word "Toronto"

In [105]:
df_toronto=df_canada[df_canada['Borough'].str.contains('Toronto')]
df_toronto=df_toronto.reset_index(drop=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [106]:
# Learn the different value of Borough 
df_toronto['Borough'].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

## visualize the Toronto data by coordinates

In [107]:
geolocator = Nominatim(user_agent="toronto_explorer")
Toronto_coord = geolocator.geocode("Toronto")
Toronto_lat = Toronto_coord.latitude
Toronto_longi = Toronto_coord.longitude
print(f'Coordinate of Toronto: {Toronto_lat}, {Toronto_longi}.')

Coordinate of Toronto: 43.6534817, -79.3839347.


## Create a clustering of Toronto

In [108]:
# define the number of max clusters
kmax =len(df_toronto.Borough.unique())

# create map
toronto_map = folium.Map(location=[Toronto_lat, Toronto_longi], zoom_start=25)

# set color scheme for the clusters
x = np.arange(kmax)
ys = [i + x + (i*x)**2 for i in range(kmax)]
colors_array = cmap.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


### create a map of Toronto first

In [109]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
toronto_map

### create DataFrame for clustering

In [110]:
df_toronto_clustering = df_toronto.drop(['Postal Code','Borough','Neighbourhood'],1).copy(deep=True)
kmeans = KMeans(n_clusters = kmax,random_state=0).fit(df_toronto_clustering)
kmeans.labels_
df_toronto_clustering.insert(0, 'Cluster Labels', kmeans.labels_)

In [111]:
# print clustering DataFrame 
df_toronto_clustering.head()

,Cluster Labels,Latitude,Longitude
0,1,43.654260,-79.360636
1,1,43.662301,-79.389494
2,1,43.657162,-79.378937
3,1,43.651494,-79.375418
4,3,43.676357,-79.293031


### Create the clustered map

In [112]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(kmax)
ys = [i + x + (i*x)**2 for i in range(kmax)]
colors_array = cmap.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df_toronto_clustering['Latitude'], df_toronto_clustering['Longitude'], df_toronto['Neighbourhood'], df_toronto_clustering['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)      
map_clusters